In [1]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
import pandas as pd
df = pd.read_csv("/content/dataset (4).csv")
df = df.rename(columns={
    "TrafficConditionNearby": "Traffic"
})

In [11]:
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                  format="%d-%m-%Y %H:%M:%S")

# Sort by Timestamp
df = df.sort_values("Timestamp").reset_index(drop=True)
df[["Timestamp", "Occupancy", "Capacity", "QueueLength", "VehicleType", "Traffic", "IsSpecialDay"]].to_csv("parking_stream.csv", index=False)

In [12]:
import pathway as pw

class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    VehicleType: str
    Traffic: float
    IsSpecialDay: int

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

In [13]:
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [14]:
import datetime

# Model 1: Baseline Linear Pricing
delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        occ_max = pw.reducers.max(pw.this.Occupancy),
        cap = pw.reducers.max(pw.this.Capacity)
    )
    # 🎯 Directly calculate price without using a temp column
    .with_columns(
        price = 10 + 10 * (pw.this.occ_max / pw.this.cap)
    )
)

In [15]:
import panel as pn
import bokeh.plotting
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=2, color="navy")
    fig.circle("t", "price", source=source, size=6, color="red")
    return fig

viz = delta_window.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [16]:
%%capture --no-display
pw.run()

Output()

In [17]:
# Constants
BASE_PRICE = 10
ALPHA = 5         # Occupancy weight
BETA = 0.1        # Queue length weight
GAMMA = 0.05      # Traffic weight (negative effect)
DELTA = 0.2       # Special day bonus
LAMBDA = 0.3      # Price responsiveness multiplier

# Vehicle type weights
VEHICLE_WEIGHTS = {
    'car': 1.0,
    'bike': 0.5,
    'truck': 1.5
}

# Demand-Based Pricing Function
def compute_model2_price(capacity, occupancy, queue_len, traffic, is_special_day, vehicle_type):
    try:
        # Calculate occupancy ratio
        occ_ratio = occupancy / capacity if capacity > 0 else 0

        # Vehicle type weight (default to 1.0 if unknown)
        vehicle_weight = VEHICLE_WEIGHTS.get(vehicle_type.lower(), 1.0)

        # Raw demand formula
        raw_demand = (
            ALPHA * occ_ratio +
            BETA * queue_len -
            GAMMA * traffic +
            DELTA * is_special_day +
            vehicle_weight
        )

        # Normalize demand
        max_possible = (
            ALPHA +          # max occ_ratio = 1
            BETA * 10 +      # assume max queue = 10
            DELTA +          # max special day = 1
            max(VEHICLE_WEIGHTS.values())  # truck = 1.5
        )
        normalized_demand = raw_demand / max_possible

        # Final price computation with bounds
        price = BASE_PRICE * (1 + LAMBDA * normalized_demand)
        price = max(BASE_PRICE * 0.5, min(price, BASE_PRICE * 2))  # Clamp between 0.5x and 2x

        return round(price, 2)

    except Exception as e:
        print("Error in pricing model:", e)
        return BASE_PRICE


In [18]:
price = compute_model2_price(
    capacity=50,
    occupancy=40,
    queue_len=5,
    traffic=0.7,
    is_special_day=1,
    vehicle_type='truck'
)
print("Model 2 Price:", price)


Model 2 Price: 12.4
